<a href="https://colab.research.google.com/github/potaHere/Submission-Klasifikasi-Gambar-Dicoding/blob/main/Template_Submission_Akhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Klasifikasi Gambar: Fruits Datasets
- **Nama:** Ja'far Shodiq
- **Email:** jafarshodiq.alkaf@gmail.com
- **ID Dicoding:** jafar_shodiq

## Import Semua Packages/Library yang Digunakan

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Preparation

### Data Loading

In [6]:
zip_path = "/content/drive/MyDrive/dataset-colab/fruit dataset.zip"  # Ubah sesuai lokasi file Anda
extract_to = "/content/drive/MyDrive/dataset-colab/fruit dataset"    # Direktori tujuan ekstraksi

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

DATASET_DIR ="/content/drive/MyDrive/dataset-colab/fruit dataset"

### Data Preprocessing

In [7]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train, 20% valid
    rotation_range=20,
    zoom_range=0.1,
    horizontal_flip=True
)

#### Split Dataset

In [ ]:
train_generator = datagen.flow_from_directory(
    extract_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    extract_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

## Modelling

In [ ]:
base_model = MobileNetV3Large(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(9, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## Evaluasi dan Visualisasi

In [ ]:
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)

### Plot Akurasi dan Loss

In [ ]:
plt.figure(figsize=(12, 5))

# Akurasi
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Konversi Model

In [ ]:
model.save("saved_model/fruit_classifier")
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model/fruit_classifier")
tflite_model = converter.convert()
with open("fruit_classifier.tflite", "wb") as f:
    f.write(tflite_model)

## Inference (Optional)